___
___
___
# Light GBM
___
___
___
## Exercice avec le dataset "wine"
___
___
___
A partir du dataset "Wine" (wine.csv), faire des prédictions sur la note à donner d'un vin (feature "points").

Le code est pré-implanté de la même manière que l'exemple ci-dessus.

Tâches à faire :
* explorer les données
* la cible étant la note donnée à un vin, elle sera considérée comme catégorielle
* afficher les résultats suivants : La matrice de confusion, Accuracy
* il faut sans doute convertir les résultats de *y_pred_lgbm*
* tester des hyperparamètres, sources : https://lightgbm.readthedocs.io/en/latest/Parameters.html | https://towardsdatascience.com/beginners-guide-to-the-must-know-lightgbm-hyperparameters-a0005a812702

### Import & données

In [13]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from datetime import datetime # juste pour chronométrer
from sklearn import metrics
from sklearn.metrics import roc_auc_score, confusion_matrix # roc_auc_score pour quantifier la matrice de confusion
# En résumé, le score AUC-ROC évalue la capacité d'un modèle à bien classer les exemples positifs par rapport
# aux exemples négatifs à différents seuils de probabilité, et il fournit un nombre unique pour mesurer cette performance,
# où un score plus élevé indique une meilleure performance.
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import lightgbm as lgb # librairie LightGBM à installer au préalable $ pip install lightgbm

df = pd.read_csv("wine.csv")

X = df.drop(labels = ["Unnamed: 0", "points"], axis=1)
y = df[['points']]

intervales = [80, 85, 90, 95, 100, 105]
etiquettes = ['80-84', '85-89', '90-94', '95-99', '100-104']

# Utiliser pd.cut pour créer une nouvelle colonne avec les catégories
y['points'] = pd.cut(y['points'], bins=intervales, labels=etiquettes, include_lowest=True)

print('##################################################')
print("Taille du dataset :", X.shape, y.shape)
print('##################################################')
y = y['points']

##################################################
Taille du dataset : (129971, 12) (129971, 1)
##################################################


/tmp/ipykernel_94858/892481712.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['points'] = pd.cut(y['points'], bins=intervales, labels=etiquettes, include_lowest=True)


### Exploration des données

In [14]:
display(X.head(2))
print("##################################################")
print("valeurs uniques de y =>", y.unique())

,country,description,designation,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos


##################################################
valeurs uniques de y => ['85-89', '80-84', '90-94', '95-99']
Categories (5, object): ['80-84' < '85-89' < '90-94' < '95-99' < '100-104']


### Nettoyage des données

In [15]:
labelencoder = LabelEncoder()

y = labelencoder.fit_transform(y)
print(y)

for column in X.columns:
    X[column] = labelencoder.fit_transform(X[column])

scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

[1 1 1 ... 1 1 1]


### Split du dataset

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
print("Taille des jeux de données :", X_train.shape, y_train.shape, X_test.shape, y_test.shape)

Taille des jeux de données : (103976, 12) (103976,) (25995, 12) (25995,)


### **LightGBM**

In [5]:
d_train = lgb.Dataset(X_train, label=y_train)

# https://lightgbm.readthedocs.io/en/latest/Parameters.html
lgbm_params = {}

start=datetime.now()
clf = lgb.train(lgbm_params, d_train, 50) #50 iterations. Increase iterations for small learning rates
stop=datetime.now()
execution_time_lgbm = stop-start

#Prediction on test data
y_pred_lgbm=clf.predict(X_test)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004020 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1997
[LightGBM] [Info] Number of data points in the train set: 103976, number of used features: 12
[LightGBM] [Info] Start training from score 1.095916


___
### Confusion matrix
___

In [6]:
plt.figure(figsize=(20, 12))
cm_lgbm = confusion_matrix(y_test, y_pred_lgbm)
sns.heatmap(cm_lgbm, annot=True)
plt.title("LightGBM")
plt.xlabel('Valeurs Prédites')
plt.ylabel('Valeurs Réelles')

################
#SUMMARY
print("################################################")
print("# LGBM execution time is: ", execution_time_lgbm)
print("################################################")
print ("# Accuracy with LGBM = ", round(metrics.accuracy_score(y_pred_lgbm,y_test) * 100, 2), '%')
print("################################################")

ValueError: Classification metrics can't handle a mix of multiclass and continuous targets

<Figure size 2000x1200 with 0 Axes>